# mesonic latency tests

This notebooks provides some code to test and adjust the latency in mesonic.

You can fill the Timeline with one of the different test cases below and see how low you can set the latency.

In [ ]:
import mesonic
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
context = mesonic.create_context()
s1i = context.synths.create("s1", mutable=False)

If we start the Playback of the Timeline we can see if we get late warnings.
They look like this
 
 `UserWarning: Playback late 0.8938464238361226`
 
 where the number is the time that the mesonic Playback is late.
 The late messages will be printed with half a secound delay so you should at most get twice every second.


You can also adjust the allowed lateness of the Playback.
Allowed lateness means that no warnings will be issued by this delay.
As normally we should have some latency added to the execution time of our TimeBundles by the BundleProcessor

In [ ]:
context.playback.allowed_lateness

In [ ]:
# If we set the allowed lateness to zero we will receive warnings very early.
context.playback.allowed_lateness = 0.0

You can adjust the BundleProcessor latency using the following code.

In [ ]:
# context.playback.processor.latency == 0.2 by default
context.playback.processor.latency

In [ ]:
context.playback.processor.latency = 0.2  

 If the lateness of the Playback exhausts the latency of the BundleProcessor and thus the time that the backend has to act on the TimeBundle.
This means if it is too low the timing will be off. You also should be able to hear this.

## Test Cases

The following cells contain different times to fill the Timeline.
Feel free to change the parameters for the data to test different settings.

In [ ]:
# function used to schedule the TimeBundles
def schedule_at(times):
    for timepoint in np.sort(times):
        with context.at(timepoint):
            s1i.start({"dur": 0.001, "amp": 0.015})

### Test Case 1: Timeline filled with equidistant times. 

In [ ]:
timebundles_per_second = 3_000
duration = 5
N = duration * timebundles_per_second  # total amount of TimeBundles

print(f"Time between TimeBundles {1/timebundles_per_second}")
print(f"Total amount of TimeBundles {N}")

context.timeline.reset()
times = np.linspace(0, duration, N)
schedule_at(times)

In [ ]:
plt.hist(times, histtype="step", bins="auto", cumulative=True);
plt.hist(times, histtype="step", bins="auto", cumulative=False);

In [ ]:
context.playback.start()

### Test Case 2: Timeline filled with one gaussian distributed peak of TimeBundles

In [ ]:
N = 4_500
duration = 5

print(f"mean TimeBundles per second {N/duration}")
print(f"mean time between TimeBundles {1/N}")

times = np.random.default_rng().normal(duration/2, duration/10, size=N)

context.timeline.reset()
schedule_at(times)

In [ ]:
plt.hist(times, histtype="step", bins="auto", cumulative=True);
plt.hist(times, histtype="step", bins="auto", cumulative=False);

In [ ]:
context.playback.start()

### Test Case 3: Timeline filled with two gaussian distributed peaks of TimeBundles

In [ ]:
N1 = 2_000
N2 = 1_000
duration = 5

print(f"mean TimeBundles per second {N/duration}")
print(f"mean time between TimeBundles {1/N}")


p1 = np.random.default_rng().normal(duration * 1/4, duration/20, size=N1)
p2 = np.random.default_rng().normal(duration * 3/4, duration/20, size=N2)
times = np.concatenate([p1, p2])

context.timeline.reset()
schedule_at(times)

In [ ]:
plt.hist(times, histtype="step", bins="auto", cumulative=True);
plt.hist(times, histtype="step", bins="auto", cumulative=False);

In [ ]:
# If we do not get warnings here the timing should be good.
context.playback.start()

In [ ]:
context.close()